# 1、Data preprocessing

In [ ]:
1.1 translate

In [ ]:
'''
    如有翻译需求，此代码可用，但需填入您的appid和secret_key
'''
import urllib.request
import urllib.parse
import json
import hashlib
import time, re
import pandas as pd
import csv

encoding = 'utf-8' # 编码用utf-8
salt = '666' #随机数
appid = #此处请填入您的百度翻译API的appid
secret_key = #此处填入您的secret_key
# 请求失败码
REQUEST_FAILED = -1

# 正则匹配
settings_regex = r"\s*\'.+\'\s*=>\s*.+"


def getMD5(content):
    m2 = hashlib.md5()
    m2.update(content.encode(encoding))
    return m2.hexdigest()


def getTranslateResponce(url, data):
    data = urllib.parse.urlencode(data).encode('utf-8')
    response = urllib.request.urlopen(url, data)
    return response.read().decode('utf-8')


def trans(content):
    url = 'http://api.fanyi.baidu.com/api/trans/vip/translate'
    data = {}
    data['appid'] = appid
    data['salt'] = salt
    data['from'] = 'auto'
    data['to'] = 'en'
    data['q'] = content
    data['sign'] = getMD5(appid + content + salt + secret_key)
    #time.sleep(1)
    html = getTranslateResponce(url, data)
    target = json.loads(html)
    
    if target.get('error_code', REQUEST_FAILED) != REQUEST_FAILED:
        print('本次请求失败，原因为：',target['error_msg'])
        return "failfailfail"
    return target['trans_result'][0]['dst']

exist = pd.read_csv(".\\Data in the analysis process\\projectFilter_translated.csv")
ok_id = exist["id"].tolist()
#print(ok_id)

f = pd.read_excel('.\\Data in the analysis processl\\projectFilter.xlsx',sheet_name='项目筛选', header=0)
wait_id = f["id"].tolist()

result = []
fr = open(".\\Data in the analysis process\\projectFilter_translated.csv", 'a+', encoding='utf-8', newline='')
csv_writer = csv.writer(fr)

need_id = set(wait_id)-set(ok_id)
print(need_id)
print(len(ok_id))
print(len(wait_id))

for line in f.values:
    #print(line[0])
    if line[0] in need_id:
        print(line[0])
        if line[19]!='en':
            if line[2]=="":
                content = "空"
            else:
                content = line[2]
            content =str(content).replace("\n", "").replace("\r", "")
            etext = trans(content)
            time.sleep(1.2)
        else:
            etext = str(line[2])
        if line[20]!='en':
            if line[9]=="":
                content1 = "空"
            else:
                content1 = line[9]
            content1 =str(content1).replace("\n", "").replace("\r", "")
            etext1 = trans(content1)
            time.sleep(1)
        else:
            etext1 = str(line[9])
        if etext=="failfailfail" or etext1=='failfailfail':
            continue;
        if etext =="nan":
            etext = ''
        if etext1 =="nan":
            etext1 = ''
        introduction = etext+" "+etext1
        csv_writer.writerow(
            [line[0], line[1], etext, line[3], line[4],line[5], line[6], line[7], line[8],etext1,line[10],
                len(introduction), introduction,line[13],line[14],line[15],line[16],line[17],line[18],line[19],line[20],line[21]])
        time.sleep(1)
    #print(etext)
fr.close()

print("Finish")

1.2 Delete stop words and other special characters except English and numbers in the text

In [1]:
import csv
import nltk
from nltk.tokenize import MWETokenizer
import re
import pandas as pd


def segWords():
    # 读取原文
    data = pd.read_csv('.\\Data in the analysis process\\projectFilter_translated_0.csv')#源文件路径（.csv）
    
    # 分词结果存放文件
    readme = pd.DataFrame(columns=data.columns.values)
    # 读取停用词
    stopList = open('.\\Data in the analysis process\\stopWords.txt', mode='r', encoding='utf-8')
    stopWords = []
    for line in stopList:
        stopWords.append(line.strip())
    if "on" in stopWords:
        print("123")

    # 清洗去停用词
    i = 0;
    print(len(data.values))
    for item in data.values:
        content = str(item[12])
        words = '';
        strinfo = re.compile('["‘\+\-\[\]_\s`~!@#$^&?*()=|{}:;,./<>\'\\\\]')
        content = strinfo.sub(' ', content)
        seg_result=content.split(" ")
        
        for word in seg_result:
            word=word.strip()
            if word.lower() in stopWords:
                continue;
            else:
                an = re.search('[`~!@#$^&*%()=|{}:;,.<>《》/?！￥…（）【】‘；:”“\'。，、？]', word)
                ann = re.search('[\u4e00-\u9fa5]',word)
                en = re.search('[\u0041-\u005A,\u0061-\u007A,0-9]',word)
                if an or ann: continue
                if not en: continue
                if word.isdigit() or word==" ": continue
                words += word.lower().strip()
                words += " "
        i = i + 1
        print(i)
        #if words != '':
        readme = readme.append([{'id':item[0],
                          'name':item[1],
                          'description':item[2],
                          'len_description':item[3],
                          'url':item[4],
                          'star':item[5],
                          'fork':item[6],
                          'issues':item[7],
                          'topic':item[8],
                          'readMe':item[9],
                          'len_readMe':item[10],
                          'length':item[11],
                          'introduction':words.strip(),
                          'owner':item[13],
                          'ownerId':item[14],
                          'ownerType':item[15],
                          'createTime':item[16],
                          'language':item[17],
                          'lang_detail':item[18],
                          'langId_description':item[19],
                          'langId_readMe':item[20],
                          'size':item[21]}], ignore_index = True)
    readme.to_csv('.\\Data in the analysis process\\projectFilter_translated_0_cleaned.csv',index = False)

segWords()

123
12199
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
27

# 2 Synonym hashtag processing

2.1 Finding Synonym hashtag

In [5]:
'''
    1、已有标签统计
'''
import numpy as np
import pandas as pd
from collections import Counter

data = pd.read_csv(".\\Data in the analysis process\\projectFilter_translated_0.csv")


all_tags = []
for line in data["topic"].values:
    if str(line) !="nan" and str(line) !="":
        tags = str(line).split(" ")
        all_tags.extend(tags)
    
result = Counter(all_tags)
d = sorted(result.items(), key=lambda x: x[1], reverse=True)


tag = pd.DataFrame({"tag":list(list(zip(*d))[0]),"count":list(list(zip(*d))[1])}, columns=["tag","count"])

tag.to_csv(".\\Data in the analysis process\\tag.csv",index = False)
print("Finish!")

Finish!


In [4]:
'''
    2、生成词向量训练语料
'''
import csv
import nltk
from nltk.tokenize import MWETokenizer
import re
import pandas as pd


def segWordsCorpus():
    # 读取词典和原文
    with open('.\\Data in the analysis process\\tag_std_0.csv', 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile);
        result = list(reader);
    data = pd.read_csv('.\\Data in the analysis process\\introduction.csv')#源文件路径（.csv）

    # 分词结果存放文件
    #readme = pd.DataFrame(columns=['id','name','description','url','star','fork','issues','topic','readMe','introduction','owner','ownerId','ownerType','createTime','language','lang_detail','langId'])
    f = open(".\\Data in the analysis process\\Word vector training corpus.txt","w",encoding="utf-8")

    # 构造词典
    corpusList = [];
    phraseList = [];
    for item in result:
        listWord = str(item[0]).split(" ");
        corpusList.append(tuple(listWord));
        phrase = listWord[0];
        i = 1
        while i < int(len(listWord)):
            phrase = phrase + '-' + listWord[i];
            i = i + 1;
        phraseList.append(phrase);

    tokenizer = MWETokenizer(corpusList, separator='-')

    # 分词（Phrase）
    i = 0;
    print(len(data.values))
    for item in data.values:
        content = str(item[0]);
        # print(content);
        seg_result = tokenizer.tokenize(nltk.word_tokenize(content.lower()));
        words = '';

        for word in seg_result:
            words += word.lower()
            words += " "
        i = i + 1
        print(i)
        f.write(words+"\n")
segWordsCorpus()

e:\python\python3.8\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


12410
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135


3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744
3745
3746
3747
3748
3749
3750
3751
3752
3753
3754
3755
3756
3757
3758
3759
3760
3761
3762
3763
3764
3765
3766
3767
3768
3769
3770
3771
3772
3773
3774
3775
3776
3777
3778
3779


5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438


6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029
7030
7031
7032
7033
7034
7035
7036
7037
7038
7039
7040
7041
7042
7043
7044
7045
7046
7047
7048
7049
7050
7051
7052
7053
7054
7055
7056
7057
7058
7059
7060
7061
7062
7063
7064
7065
7066
7067
7068
7069
7070
7071
7072
7073
7074
7075
7076
7077
7078
7079
7080
7081
7082
7083
7084
7085
7086
7087
7088
7089
7090
7091
7092
7093
7094
7095
7096
7097
7098
7099
7100
7101
7102
7103
7104
7105
7106
7107
7108


8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8670
8671
8672
8673
8674
8675
8676
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8688
8689
8690
8691
8692
8693
8694
8695
8696
8697
8698
8699
8700
8701
8702
8703
8704
8705
8706
8707
8708
8709
8710
8711
8712
8713
8714
8715
8716
8717
8718
8719
8720
8721
8722
8723
8724
8725
8726
8727
8728
8729
8730
8731
8732
8733
8734
8735
8736
8737
8738
8739
8740
8741
8742
8743
8744
8745
8746
8747
8748
8749
8750
8751
8752
8753
8754
8755
8756
8757
8758
8759
8760
8761
8762
8763
8764
8765
8766
8767
8768
8769
8770
8771
8772
8773
8774
8775
8776
8777
8778
8779
8780
8781
8782
8783
8784
8785
8786
8787
8788
8789
8790
8791
8792
8793
8794
8795
8796
8797
8798
8799
8800
8801
8802
8803
8804
8805
8806
8807
8808
8809
8810
8811
8812
8813
8814
8815
8816
8817
8818
8819
8820
8821
8822
8823
8824
8825
8826
8827
8828
8829
8830
8831
8832
8833
8834
8835
8836
8837
8838
8839
8840
8841
8842
8843
8844
8845
8846
8847
8848
8849
8850
8851
8852
8853
8854
8855
8856
8857
8858


10285
10286
10287
10288
10289
10290
10291
10292
10293
10294
10295
10296
10297
10298
10299
10300
10301
10302
10303
10304
10305
10306
10307
10308
10309
10310
10311
10312
10313
10314
10315
10316
10317
10318
10319
10320
10321
10322
10323
10324
10325
10326
10327
10328
10329
10330
10331
10332
10333
10334
10335
10336
10337
10338
10339
10340
10341
10342
10343
10344
10345
10346
10347
10348
10349
10350
10351
10352
10353
10354
10355
10356
10357
10358
10359
10360
10361
10362
10363
10364
10365
10366
10367
10368
10369
10370
10371
10372
10373
10374
10375
10376
10377
10378
10379
10380
10381
10382
10383
10384
10385
10386
10387
10388
10389
10390
10391
10392
10393
10394
10395
10396
10397
10398
10399
10400
10401
10402
10403
10404
10405
10406
10407
10408
10409
10410
10411
10412
10413
10414
10415
10416
10417
10418
10419
10420
10421
10422
10423
10424
10425
10426
10427
10428
10429
10430
10431
10432
10433
10434
10435
10436
10437
10438
10439
10440
10441
10442
10443
10444
10445
10446
10447
10448
10449
10450
1045

11770
11771
11772
11773
11774
11775
11776
11777
11778
11779
11780
11781
11782
11783
11784
11785
11786
11787
11788
11789
11790
11791
11792
11793
11794
11795
11796
11797
11798
11799
11800
11801
11802
11803
11804
11805
11806
11807
11808
11809
11810
11811
11812
11813
11814
11815
11816
11817
11818
11819
11820
11821
11822
11823
11824
11825
11826
11827
11828
11829
11830
11831
11832
11833
11834
11835
11836
11837
11838
11839
11840
11841
11842
11843
11844
11845
11846
11847
11848
11849
11850
11851
11852
11853
11854
11855
11856
11857
11858
11859
11860
11861
11862
11863
11864
11865
11866
11867
11868
11869
11870
11871
11872
11873
11874
11875
11876
11877
11878
11879
11880
11881
11882
11883
11884
11885
11886
11887
11888
11889
11890
11891
11892
11893
11894
11895
11896
11897
11898
11899
11900
11901
11902
11903
11904
11905
11906
11907
11908
11909
11910
11911
11912
11913
11914
11915
11916
11917
11918
11919
11920
11921
11922
11923
11924
11925
11926
11927
11928
11929
11930
11931
11932
11933
11934
11935
1193

In [9]:
'''
    3、使用gensim的word2vec模型训练词向量
'''
import gensim
import codecs
from gensim.models import word2vec
import re
from gensim.corpora.dictionary import Dictionary

import pickle
import logging

import numpy as up

#引入日志配置

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus('.\\Data in the analysis process\\Word vector training corpus.txt')
model=word2vec.Word2Vec(sentences,size=128,iter=5,workers=3,min_count=1)
model.save('word2vecAll.model')
print(model.similarity('javascript','js'))


###将模型保存
model.wv.save_word2vec_format('.\\Data in the analysis process\\word2vecAll.vector')
model.save('.\\Data in the analysis process\\word2vecAll.model')
print(model["java"])

2021-02-05 12:18:46,270 : INFO : collecting all words and their counts
2021-02-05 12:18:46,287 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-02-05 12:18:47,365 : INFO : collected 172320 word types from a corpus of 4388025 raw words and 439 sentences
2021-02-05 12:18:47,366 : INFO : Loading a fresh vocabulary
2021-02-05 12:18:47,825 : INFO : effective_min_count=1 retains 172320 unique words (100% of original 172320, drops 0)
2021-02-05 12:18:47,826 : INFO : effective_min_count=1 leaves 4388025 word corpus (100% of original 4388025, drops 0)
2021-02-05 12:18:48,262 : INFO : deleting the raw counts dictionary of 172320 items
2021-02-05 12:18:48,265 : INFO : sample=0.001 downsamples 36 most-common words
2021-02-05 12:18:48,266 : INFO : downsampling leaves estimated 3378850 word corpus (77.0% of prior 4388025)
2021-02-05 12:18:48,577 : INFO : estimated required memory for 172320 words and 128 dimensions: 262615680 bytes
2021-02-05 12:18:48,578 : INFO : rese

0.8641607


2021-02-05 12:19:54,966 : INFO : saving Word2Vec object under .\Data in the analysis process\word2vecAll.model, separately None
2021-02-05 12:19:54,967 : INFO : storing np array 'vectors' to .\Data in the analysis process\word2vecAll.model.wv.vectors.npy
2021-02-05 12:19:55,162 : INFO : not storing attribute vectors_norm
2021-02-05 12:19:55,162 : INFO : storing np array 'syn1neg' to .\Data in the analysis process\word2vecAll.model.trainables.syn1neg.npy
2021-02-05 12:19:55,371 : INFO : not storing attribute cum_table
2021-02-05 12:19:55,611 : INFO : saved .\Data in the analysis process\word2vecAll.model


[-0.24692702 -0.20902961  0.55095047 -1.5655481   0.5873085   0.21111716
 -0.24377497  1.4624377   0.55196816 -0.6822052  -0.5391541  -1.0630872
  0.6986778   1.7394378   1.471879    0.15467894 -0.53292304  0.8423998
 -0.40019342  1.2702183  -0.05555252 -0.4347502  -0.79067713  0.21773039
 -0.33257228  0.9814147   0.43861955 -0.7968793   1.3727998  -0.19789296
 -0.5033197   0.2814243  -1.2812264  -0.3859742   0.26378953  0.11831662
 -0.6536806   0.707594    0.36038747 -0.8205805   0.12016138 -0.25498605
  0.3430097   0.27156022  0.76503426 -1.0212269  -0.41165438 -0.6455671
  0.1740614  -0.44899338 -0.42606324 -0.28792834 -1.4493438   0.81373477
  0.39879847 -0.22665879 -1.3071823  -0.26341605  1.1257126   0.5967509
 -1.9709172   1.2213304   0.7690131   0.4859075   0.6702445  -0.46840945
 -0.46009773  0.03760823 -0.00580679  1.2073765  -0.17740376 -0.02276219
  1.9762356  -0.46316347  0.7636079   2.19638     1.2448478  -0.7483598
  0.2999666  -0.19951719  0.3930305   0.64701384 -1.1745

<ipython-input-9-29a62c75bde0>:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  print(model["java"])


In [25]:
'''
    4、获取每个标签语义最相似的10个主题标签，以及编辑距离词长五分之一的主题标签
'''

# encoding: utf-8
import gensim
from gensim.models import word2vec
from gensim.corpora.dictionary import Dictionary
import pandas as pd
import sys
import numpy as np

def edit_distance(word1, word2):
    len1 = len(word1)
    len2 = len(word2)
    dp = np.zeros((len1 + 1,len2 + 1))
    for i in range(len1 + 1):
        dp[i][0] = i    
    for j in range(len2 + 1):
        dp[0][j] = j
     
    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            delta = 0 if word1[i-1] == word2[j-1] else 1
            dp[i][j] = min(dp[i - 1][j - 1] + delta, min(dp[i-1][j] + 1, dp[i][j - 1] + 1))
    return dp[len1][len2]


#加载训练好的模型
model = word2vec.Word2Vec.load('.\\Data in the analysis process\\word2vecAll.model')


word_dic = pd.read_csv(".\\Data in the analysis process\\tag.csv")

result_df = pd.DataFrame(columns=["tag1","tag2","s"])

tags = word_dic["tag"].values

for i in range(0,len(tags)-1):
    print(i)
    for j in range(i+1,len(tags)):
        if edit_distance(tags[i],tags[j])/len(tags[i])<0.2:
            result_df=result_df.append([{"tag1":tags[i],"tag2":tags[j]}],ignore_index=True)

#print(model.wv.vocab)
vocab = model.wv.vocab

for i in range(0,len(tags)):
    if tags[i] in vocab:
        res = model.most_similar(tags[i],topn=10)
        for j in res:
            if j[0] in tags and j[1]>0.5:
                result_df=result_df.append([{"tag1":tags[i],"tag2":j[0],"s":j[1]}],ignore_index=True)
        print(i)
result_df.to_csv(".\\Data in the analysis process\\Most_similar_words.csv")

2020-11-24 11:17:18,712 : INFO : loading Word2Vec object from .\Data in the analysis process\word2vecAll.model
2020-11-24 11:17:18,797 : INFO : loading wv recursively from .\Data in the analysis process\word2vecAll.model.wv.* with mmap=None
2020-11-24 11:17:18,797 : INFO : setting ignored attribute vectors_norm to None
2020-11-24 11:17:18,813 : INFO : loading vocabulary recursively from .\Data in the analysis process\word2vecAll.model.vocabulary.* with mmap=None
2020-11-24 11:17:18,813 : INFO : loading trainables recursively from .\Data in the analysis process\word2vecAll.model.trainables.* with mmap=None
2020-11-24 11:17:18,813 : INFO : setting ignored attribute cum_table to None
2020-11-24 11:17:18,813 : INFO : loaded .\Data in the analysis process\word2vecAll.model


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

<ipython-input-25-aa7d823aa1f6>:50: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  res = model.most_similar(tags[i],topn=10)
2020-11-24 12:37:08,604 : INFO : precomputing L2-norms of word weight vectors


5671
5672
5673
5674
5675
5676
5677
5678
5679
5680
5681
5682
5683
5684
5685
5686
5687
5688
5689
5690
5691
5692
5693
5694
5695
5696
5697
0
1
2
3
4
5
6
7
8
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
137
138
139
140
141
142
143
144
145
146
148
150
151
152
153
156
157
158
159
160
161
162
163
164
165
166
167
169
170
171
172
173
174
177
178
179
181
182
183
184
185
186
188
189
190
191
192
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
220
221
222
223
224
225
226
229
230
231
232
233
234
235
236
237
238
239
240
241
242
244
245
246
247
248
249
250
251
252
253
254
255
257
258
259
260
262
263
26

The generated similar words are filtered manually to obtain a synonym dictionary
(Thesaurus.csv)

2.2 Use the synonym dictionary to replace the synonym hashtags in the original hashtags

In [11]:
import pandas as pd

data = pd.read_csv(".\\Data in the analysis process\\projectFilter_translated_0_cleaned.csv")
sametag = pd.read_csv(".\\Data in the analysis process\\Thesaurus.csv")
yuanshitag = pd.read_csv(".\\Data in the analysis process\\tag.csv")
tagset = yuanshitag["tag"].values

sametagdic = dict(zip(sametag["label"].values, sametag["replace by"].values))

data_std = pd.DataFrame()

for item in data.values:
    tags = str(item[8]).split(" ")
    #if str(line[4]) != 'nan':
    #    tags.append(line[4].lower().replace(" ",""))
    for i in tags:
        if i not in tagset: 
            tags.remove(i)
            continue
        if i in sametagdic.keys():
            tags[tags.index(i)] = sametagdic[i]
    for i in tags:
        tags[tags.index(i)] = tags[tags.index(i)].replace("-"," ")
    tags_std = list(set(tags))
    #data_std = data_std.append([{'url':line[0],'topic':";".join(tags_std),'introduction':line[2],'createTime':str(line[3])[:10],'lang':line[4],'tagNum':len(tags_std),'star':line[5]}],ignore_index=True)
    data_std = data_std.append([{
        'id':item[0],
        'name':item[1],
        'description':item[2],
          'len_description':item[3],
          'url':item[4],
          'star':item[5],
          'fork':item[6],
          'issues':item[7],
          'topic':";".join(tags_std),
          'readMe':item[9],
          'len_readMe':item[10],
          'length':item[11],
          'introduction':item[12],
          'owner':item[13],
          'ownerId':item[14],
          'ownerType':item[15],
          'createTime':item[16],
          'language':item[17],
          'lang_detail':item[18],
          'langId_description':item[19],
          'langId_readMe':item[20],
          'size':item[21]}],ignore_index=True)

data_std.to_csv(".\\Data in the analysis process\\projectFilter_translated_0_cleaned_std.csv",index=False)
print("Finish!")

Finish!


2.3 Select core hashtag

In [12]:
'''
   统计出经过同义替换的主题标签以及词频，进而进行核心主题标签的选择
'''
import numpy as np
import pandas as pd
from collections import Counter

data = pd.read_csv(".\\Data in the analysis process\\projectFilter_translated_0_cleaned_std.csv")


all_tags = []
for line in data["topic"].values:
    if str(line) !="nan" and str(line) !="":
        tags = str(line).split(";")
        all_tags.extend(tags)
    
result = Counter(all_tags)
d = sorted(result.items(), key=lambda x: x[1], reverse=True)


tag = pd.DataFrame({"tag":list(list(zip(*d))[0]),"count":list(list(zip(*d))[1])}, columns=["tag","count"])

tag.to_csv(".\\Data in the analysis process\\tag_std_0.csv",index = False)
print("Finish!")

Finish!


Calculate the proportion of cumulative word frequency in Excel and select the core hashtags

In [13]:
'''
    标签选择
'''
import pandas as pd

data = pd.read_csv(".\\Data in the analysis process\\projectFilter_translated_0_cleaned_std.csv")
yuanshitag = pd.read_csv(".\\Data in the analysis process\\frequent_tag.csv")
tagset = yuanshitag["tag"].values


data_std = pd.DataFrame()

for item in data.values:
    tags = str(item[8]).split(";")
    
    tags_std = []
    for i in tags:
        if i in tagset: 
            tags_std.append(i)
            
    data_std = data_std.append([{
        'id':item[0],
        'name':item[1],
        'description':item[2],
          'len_description':item[3],
          'url':item[4],
          'star':item[5],
          'fork':item[6],
          'issues':item[7],
          'topic':";".join(tags_std),
          'readMe':item[9],
          'len_readMe':item[10],
          'length':item[11],
          'introduction':item[12],
          'owner':item[13],
          'ownerId':item[14],
          'ownerType':item[15],
          'createTime':item[16],
          'language':item[17],
          'lang_detail':item[18],
          'langId_description':item[19],
          'langId_readMe':item[20],
          'size':item[21]}],ignore_index=True)

data_std.to_csv(".\\Data in the analysis process\\projectFilter_translated_0_cleaned_std_selected.csv",index=False)
print("Finish!")

Finish!


After the above steps, the preprocessing and the selection of the core hashtags are all completed. The final generated file is“projectFilter_translated_0_cleaned_std_selected.csv”

# 3、标签抽取

3.1 Text Vectorization

In [1]:
'''
文本向量化
'''
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2 # 卡方检验
import numpy as np
import pickle
from sklearn import preprocessing

data_df=pd.read_csv(".\\Data in the analysis process\\projectFilter_translated_0_cleaned_std_selected.csv")#源文件路径（.csv）
#data_df=data_df[~(data_df['introduction'].isnull())] #删掉空行
#data_df=data_df[~(data_df['topic'].isnull())] #删掉空行

#读取数据
introduction_list = data_df["introduction"].values.tolist()
url_list = data_df["url"].values.tolist()

train_data_index = data_df[(data_df["tag_count"])>0].index.tolist()
wait_data_index = data_df[(data_df["tag_count"])<=0].index.tolist()

#文本向量化
vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1, 1),
                             token_pattern = r"\b[\u0041-\u005A,\u0061-\u007A,0-9]{0,1}[\u0041-\u005A,\u0061-\u007A]{1,20}[\u0041-\u005A,\u0061-\u007A,-,0-9]{0,2}\b",
                             analyzer = "word",
                             max_features=5000)
introduction_matrix=vectorizer.fit_transform(introduction_list)
r_m = introduction_matrix.todense()

#区分训练数据和待抽取标签数据
train_data = []
wait_data = []
train_label = []
for index in train_data_index:
    train_data.append(np.array(r_m[index])[0])
    train_label.append(data_df.iloc[index,8])
for index in wait_data_index:
    wait_data.append(np.array(r_m[index])[0])
train_data = np.array(train_data)
wait_data = np.array(wait_data)

#统计标签的函数
def count_topic(topics):
    topic_set=[]
    for i in topics:
        for j in i.split(";"):
            topic_set.append(j)
    topic = [i for i in topic_set if topic_set.count(i)>0]
    return list(set(topic))

#构造训练label
tags = []
topics = count_topic(train_label)
length = len(topics)
for i in train_label:#data_df["topic"].values.tolist():
    tag = np.zeros(length)
    for j in i.split(";"):
        if j in topics:
            tag[topics.index(j)]=1
    tags.append(tag)
tags = np.array(tags)
print(tags)
print(length)

#特征选择，卡方
features = []
for i in range(len(tags[0])):
    target = tags[:,i]
    model1 = SelectKBest(chi2, k=30)#选择k个最佳特征
    model1=model1.fit(train_data,target)#data是特征数据，target是标签数据
    #print(model1.get_support())
    features.append(model1.get_support())

#将选出的特征转换为索引
features=np.array(features)
feature = []
for i in range(len(features[0])):
    f = features[:,i]
    if True in f:
        feature.append(i)
    
#保存特征索引
np.savez_compressed(".\\Data in the analysis process\\feature_index",wait_data)

#选出特征列的值构成训练数据
feature_names=vectorizer.get_feature_names()
feature_name = []
ff = []
wait_f = []
for i in feature:
    feature_name.append(feature_names[i])
    ff.append(train_data[:,i])
    wait_f.append(wait_data[:,i])
print(feature_name)
ff = np.array(ff)
ff = ff.transpose()
wait_f = np.array(wait_f)
wait_f = wait_f.transpose()

#数据标准化
scaler = preprocessing.MinMaxScaler()
ff = scaler.fit_transform(ff)
print(ff)

#将向量化的待抽取标签数据保存
np.savez_compressed(".\\Data in the analysis process\\wait_data",wait_f)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 1.]]
107
['3d', '3rd', 'abc', 'abide', 'abs', 'absence', 'absolute', 'academy', 'accounting', 'accuracy', 'accurately', 'activate', 'active', 'activecases', 'adjustment', 'admin', 'administrator', 'admission', 'admitted', 'adobe', 'adult', 'advanced', 'advertising', 'affairs', 'affinity', 'africa', 'african', 'agency', 'agent', 'agents', 'agreed', 'ai', 'air', 'airflow', 'airport', 'ajax', 'alert', 'alex', 'algorithms', 'alias', 'aligned', 'alignment', 'allocation', 'allowed', 'alpha', 'alternatively', 'america', 'analysing', 'analysis', 'analytics', 'analyzer', 'android', 'angular', 'animated', 'animations', 'annotations', 'announcement', 'announcements', 'anonymized', 'answering', 'antiviral', 'apache', 'api', 'apk', 'app', 'apple', 'application', 'applications', 'arabia', 'arabic', 'architecture', 'archive', 'arduino', 'areas', 'argen

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.11264009 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.50499481 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [2]:
'''
    查看特征数量
'''
print(len(feature_name))

1689


In [3]:
'''
    查看训练集平均标签数
    查看训练集标签数的中值
    查看训练集只有一个标签的项目有多少个
'''

aa= []
for line in tags:
    aa.append(list(line).count(1))
aa=np.array(aa)
print(np.mean(aa))
print(np.median(aa))
print((list(aa).count(1))/len(tags))

2.9574525020069573
3.0
0.23414503612523416


3.2 model select

In [4]:
'''
    测试模型p,r
'''
'''
def test_p(predictions,y_test):
    P = []
    R = []
    L = []
    for i,j in zip(predictions,y_test):
        TP = 0
        FP = 0
        FN = 0
        for m,n in zip(i,j):
            if m==1 and n==1:
                TP+=1
            if m==1 and n==0:
                FP+=1
            if m==0 and n==1:
                FN+=1
        if TP+FP!=0:
            P.append(TP/(TP+FP))
            R.append(TP/(TP+FN))
        else:
            P.append(0)
        L.append(TP+FP)
    return np.mean(P),np.mean(L),np.mean(R)
'''

from sklearn import metrics

def test_p(predictions,y_test):
    p = metrics.precision_score(y_test, predictions, average='samples')
    r = metrics.recall_score(y_test, predictions, average='samples')
    f1 = metrics.f1_score(y_test, predictions, average='samples')
    
    return p,r,f1

3.2.1 KNN

In [5]:
'''
    调参
'''
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import GridSearchCV

#KNeighborsClassifier 参数调优
param_test1 = {
#   'n_neighbors':[i for i in range(3,21)]}
#     'weights':['uniform', 'distance']}
#     'p':[1,2,3]}
score = 'f1_samples'

gsearch1 = GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=3,weights='distance',p=3), 
                        param_grid = param_test1,
                        n_jobs=-1,
                        scoring=score,
                        cv=5)
gsearch1.fit(np.array(ff),tags)
print(gsearch1.best_params_)

KeyboardInterrupt: 

In [9]:
'''
    十折交叉验证KNN
'''
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import KFold


#十折交叉验证
P = []
R = []
F = []
def ten_flod(ff,tags,P,R,F):
    kf = KFold(n_splits=10,shuffle=True,random_state=15).split(ff)
    for train_idx, valid_idx in kf:
        X_train = ff[train_idx]
        y_train = tags[train_idx]
        X_test = ff[valid_idx]
        y_test = tags[valid_idx]
        model = KNeighborsClassifier(n_neighbors=3,weights='distance',p=3)
        print(len(train_idx),len(valid_idx))
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        p,r,f1 = test_p(predictions,y_test)
        print(p,r,f1)
        P.append(p)
        # L.append(l)
        R.append(r)
        F.append(f1)
    return np.mean(P),np.mean(R),np.mean(F)

p,r,f = ten_flod(np.array(ff),tags,P,R,F)
print("KNN十折交叉验证结果：P:%.03f,R:%.03f,F:%.03f"%(p,r,f))

3363 374


e:\python\python3.8\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8057486631016043 0.4924797053807749 0.5540951175977914
3363 374


e:\python\python3.8\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8237012987012986 0.462510610304728 0.5417411055268871
3363 374


...

e:\python\python3.8\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.7930741733690797 0.4757841726340386 0.5446428311039571
3364 373


e:\python\python3.8\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8001117068811439 0.4827835033599108 0.548677313154525
3364 373
0.8149687220732797 0.4644360108167079 0.5384385051945374
KNN十折交叉验证结果：P:0.798,R:0.468,F:0.537


e:\python\python3.8\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3.2..2 随机森林

In [21]:
'''
    调参
'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#参数调优
parameters={#"n_estimators": [i*10 for i in range(2,25)]} 
#             "criterion": ["gini","entropy"]} 
#             "max_depth": [i*10 for i in range(1,11)]}
#              "max_features":[i*0.1 for i in range(5,11)]}
#             "oob_score": ["True","False"]}
svc = GridSearchCV(RandomForestClassifier(class_weight="balanced",
                                          n_estimators=40,
                                          criterion="entropy",
                                         max_depth=80,
                                         max_features=1,
                                         oob_score = True), 
                   param_grid=parameters,
                   n_jobs=-1, 
                   cv=5)#5折交叉验证
svc.fit(ff,tags)
print('The parameters of the best model are: ')
print(svc.best_params_)

The parameters of the best model are: 
{'oob_score': 'True'}


In [12]:
'''
    十折交叉验证随机森林
'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold


#十折交叉验证
P = []
R = []
F = []
def ten_flod(ff,tags,P,R,F):
    kf = KFold(n_splits=10,shuffle=True,random_state=15).split(ff)
    for train_idx, valid_idx in kf:
        X_train = ff[train_idx]
        y_train = tags[train_idx]
        X_test = ff[valid_idx]
        y_test = tags[valid_idx]
        model = RandomForestClassifier(class_weight="balanced",
                                          n_estimators=40,
                                          criterion="entropy",
                                         max_depth=80,
                                         max_features=1,
                                         oob_score = True)
        print(len(train_idx),len(valid_idx))
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        p,r,f1 = test_p(predictions,y_test)
        print(p,r,f1)
        P.append(p)
        # L.append(l)
        R.append(r)
        F.append(f1)
    return np.mean(P),np.mean(R),np.mean(F)

p,r,f = ten_flod(np.array(ff),tags,P,R,F)
print("随机森林十折交叉验证结果：P:%.03f,R:%.03f,F:%.03f"%(p,r,f))

3363 374
0.5157499363381717 0.5282475249052789 0.4343959464044397
3363 374


e:\python\python3.8\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.8351158645276292 0.4250912486206604 0.513410965148933
3363 374
0.738572701807996 0.44502376708259056 0.5014866001497017
3363 374
0.8801247771836006 0.4436594748092074 0.5430845892075838
3363 374
0.7318181818181818 0.4618475627165467 0.5072824382984811
3363 374
0.7174051438757321 0.4374936338171632 0.4842369899856531
3363 374


e:\python\python3.8\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.7680035650623886 0.41439361292302473 0.48468373575325446
3364 373
0.8127792672028596 0.43858046183783445 0.5152090834396463
3364 373
0.7014298480786416 0.48115142230423724 0.5140641009273716
3364 373
0.7999553172475424 0.44357545195346804 0.5149968292059444
随机森林十折交叉验证结果：P:0.750,R:0.452,F:0.501


3.2.3 SVM

In [3]:
'''
    调参
'''

from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

#svm调参
# parameters={#"estimator__C": np.logspace(-4, 4, 9)}
#             "estimator__kernel": ['linear','poly','rbf','sigmoid']}
#             #"estimator__gamma": np.logspace(-4, 4,9)}
# score = 'f1_macro'
# svc = GridSearchCV(OneVsRestClassifier(SVC(class_weight="balanced")), 
#                    param_grid=parameters,
#                    n_jobs=6, 
#                    scoring=score,
#                    cv=5)#5折交叉验证
# svc.fit(np.array(ff),tags)
# print('The parameters of the best model are: ')
# print(svc.best_params_)

In [4]:
'''
    十折交叉验证SVM
'''
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold


#十折交叉验证
P = []
R = []
F = []
def ten_flod(ff,tags,P,R,F):
    kf = KFold(n_splits=10,shuffle=True,random_state=15).split(ff)
    for train_idx, valid_idx in kf:
        X_train = ff[train_idx]
        y_train = tags[train_idx]
        X_test = ff[valid_idx]
        y_test = tags[valid_idx]
        model = OneVsRestClassifier(SVC(class_weight="balanced",kernel="rbf"))
        print(len(train_idx),len(valid_idx))
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        p,r,f1 = test_p(predictions,y_test)
        print(p,r,f1)
        P.append(p)
        # L.append(l)
        R.append(r)
        F.append(f1)
    return np.mean(P),np.mean(R),np.mean(F)

p,r,f = ten_flod(np.array(ff),tags,P,R,F)
print("SVR十折交叉验证结果：P:%.03f,R:%.03f,F:%.03f"%(p,r,f))

3363 374
0.7548565293886149 1.9946524064171123 0.5635287172720328 0.6453096055318319
3363 374
0.7252567693744164 2.2620320855614975 0.5319545512575002 0.6137450927583347
3363 374
0.7052781953812153 2.1844919786096257 0.5109410430839002 0.5925832537679347
3363 374
0.7089922180268367 2.090909090909091 0.5434023206625945 0.6152502341804638
3363 374
0.743669892199304 2.021390374331551 0.5680402406817501 0.6440972192916194
3363 374
0.7287477373144414 2.2566844919786098 0.5377026796381635 0.6188155586495229
3363 374
0.739925805032757 2.0454545454545454 0.507583500728662 0.6021183628265891
3364 373
0.7152381726108268 2.1849865951742626 0.5358441751873387 0.6126794280802653
3364 373
0.7307142663711028 2.1849865951742626 0.5770435105555748 0.6448501900263806
3364 373
0.7362939934655752 2.2037533512064345 0.5328491888464935 0.6182653976928609
SVR十折交叉验证结果：P:0.729,L:2.143,R:0.541,F:0.621


3.2.4 MultinomialNB

In [6]:
'''
    调参
'''
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier

#MultinomialNB调参
parameters={"estimator__alpha": [0,0.2,0.4,0.6,0.8,1,1.2,1.5]}
score = 'f1_macro'

NB = GridSearchCV(OneVsRestClassifier(MultinomialNB()), 
                   param_grid=parameters,
                   n_jobs=6, 
                   scoring=score,
                   cv=5)#5折交叉验证
NB.fit(np.array(ff),tags)
print('The parameters of the best model are: ')
print(NB.best_params_)

The parameters of the best model are: 
{'estimator__alpha': 0.2}


In [15]:
'''
    十折交叉验证MultinomialNB
'''
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold


#十折交叉验证
P = []
L = []
R = []
F = []
def ten_flod(ff,tags,P,L,R,F):
    kf = KFold(n_splits=10,shuffle=True,random_state=15).split(ff)
    for train_idx, valid_idx in kf:
        X_train = ff[train_idx]
        y_train = tags[train_idx]
        X_test = ff[valid_idx]
        y_test = tags[valid_idx]
        model = OneVsRestClassifier(MultinomialNB(alpha=0.2))
        print(len(train_idx),len(valid_idx))
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        p,l,r = test_p(predictions,y_test)
        print(p,l,r,2*p*r/(p+r))
        P.append(p)
        L.append(l)
        R.append(r)
        F.append(2*p*r/(p+r))
    return np.mean(P),np.mean(L),np.mean(R),np.mean(2*np.mean(P)*np.mean(R)/(np.mean(P)+np.mean(R)))

p,l,r,f = ten_flod(np.array(ff),tags,P,L,R,F)
print("MultinomialNB十折交叉验证结果：P:%.03f,L:%.03f,R:%.03f,F:%.03f"%(p,l,r,f))

3363 374
0.7264323487394534 3.157754010695187 0.5829148050404735 0.6468080977884407
3363 374
0.7072502373624207 2.8850267379679146 0.5678688141923437 0.6299417346773165
3363 374
0.6881894768926855 2.4946524064171123 0.5135505766202817 0.5881806163473025
3363 374
0.7050621786393815 3.0401069518716577 0.5882648778091137 0.6413896845304892
3363 374
0.6910542801192378 2.9224598930481283 0.5931157775915841 0.6383503403615606
3363 374
0.7051522785040147 3.0401069518716577 0.5601986249045073 0.6243727897172793
3363 374
0.7287428409638361 2.7967914438502675 0.5338760912290325 0.6162720509333905
3364 373
0.6845332744942021 3.131367292225201 0.5570781742152711 0.6142639021912903
3364 373
0.7294030887194425 2.418230563002681 0.5627975465106831 0.6353599550359454
3364 373
0.7124347134755588 2.648793565683646 0.5410102367763658 0.615000240653006
MultinomialNB十折交叉验证结果：P:0.708,L:2.854,R:0.560,F:0.625


3.2.5 LogisticRegression

In [9]:
'''
    调参
'''
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier

#LogisticRegression调参
parameters={#"estimator__penalty": ['l1','l2','elasticnet','none']}
            #"estimator__C":np.logspace(-4, 4, 9)}
score = 'f1_macro'

LR = GridSearchCV(OneVsRestClassifier(LogisticRegression(class_weight="balanced")), 
                   param_grid=parameters,
                   n_jobs=6, 
                   scoring=score,
                   cv=5)#5折交叉验证
LR.fit(np.array(ff),tags)
print('The parameters of the best model are: ')
print(LR.best_params_)

The parameters of the best model are: 
{'estimator__C': 1.0}


In [10]:
'''
    十折交叉验证LogisticRegression
'''
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold


#十折交叉验证
P = []
L = []
R = []
F = []
def ten_flod(ff,tags,P,L,R,F):
    kf = KFold(n_splits=10,shuffle=True,random_state=15).split(ff)
    for train_idx, valid_idx in kf:
        X_train = ff[train_idx]
        y_train = tags[train_idx]
        X_test = ff[valid_idx]
        y_test = tags[valid_idx]
        model = OneVsRestClassifier(LogisticRegression(class_weight="balanced"))
        print(len(train_idx),len(valid_idx))
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        p,l,r = test_p(predictions,y_test)
        print(p,l,r,2*p*r/(p+r))
        P.append(p)
        L.append(l)
        R.append(r)
        F.append(2*p*r/(p+r))
    return np.mean(P),np.mean(L),np.mean(R),np.mean(2*np.mean(P)*np.mean(R)/(np.mean(P)+np.mean(R)))

p,l,r,f = ten_flod(np.array(ff),tags,P,L,R,F)
print("LogisticRegression十折交叉验证结果：P:%.03f,L:%.03f,R:%.03f,F:%.03f"%(p,l,r,f))

3363 374
0.4890437984650001 4.4411764705882355 0.6482187466058433 0.5574919519689198
3363 374
0.4903825771236907 4.863636363636363 0.6463365849484448 0.5576614009718436
3363 374
0.4308216444842638 5.109625668449198 0.6154975251749445 0.506861890064064
3363 374
0.4422615449019192 5.018716577540107 0.6616386841386841 0.5301517998023433
3363 374
0.48195695050641574 4.719251336898396 0.6543938616041041 0.5550920835495599
3363 374
0.4657344152557394 4.810160427807487 0.6297114972114972 0.5354501077521422
3363 374
0.4624059021786294 4.708556149732621 0.6164164762955086 0.5284180649688467
3364 373
0.46005753826681123 4.879356568364611 0.6409926841378455 0.5356586108623734
3364 373
0.4514357254303636 4.911528150134048 0.6415657540657541 0.5299639699142433
3364 373
0.44522177178449224 4.7801608579088475 0.6188462245372814 0.5178688082022043
LogisticRegression十折交叉验证结果：P:0.462,L:4.824,R:0.637,F:0.536


3.3 Hashtag extraction

In [16]:
'''
    模型训练
'''
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

model = OneVsRestClassifier(OneVsRestClassifier(SVC(class_weight="balanced",kernel="rbf")),
                                    n_jobs=-1)
model.fit(ff,tags)

OneVsRestClassifier(estimator=OneVsRestClassifier(estimator=SVC(class_weight='balanced')),
                    n_jobs=-1)

In [18]:
data_source=pd.read_csv(".\\Data in the analysis process\\projectFilter_translated_0_cleaned_std_selected.csv")#源文件路径（.csv）
values = data_source.values

for i in range(len(wait_data_index)):
    index = wait_data_index[i]
    print(index)
    predictions = model.predict(np.array([wait_f[i]]))
    r = []
    for j in range(len(predictions[0])):
        if predictions[0][j] == 1:
            r.append(topics[j])
            
    #print(data_source.iloc[int(index),8])
    values[index,8] = ";".join(r)
    print(";".join(r)) 
OK = pd.DataFrame(values,columns=list(data_source.columns))
OK.to_csv(".\\Data in the analysis process\\extractionResult_SVM.csv",index=False)

13
react;covid 19
14
covid 19
15
covid 19
16
covid 19
17
covid 19
19
covid 19
21
covid 19
22
covid 19
24
covid 19
25
covid 19
26
covid 19
29
covid19 data;covid 19
31
covid 19
34
covid 19
35
covid 19
36
covid 19
37
covid 19
38
covid 19
40
covid 19
41
covid 19
46
covid 19
47
covid 19
48
covid 19
49
covid 19
51
covid 19
53
covid 19
55
covid 19
56
covid19 data;covid 19
63
coronavirus tracking;flutter apps;flutter;covid 19;dart
65
covid 19
66
covid 19
69
covid 19
70
covid 19
71
covid 19
73
covid 19
76
covid 19
77
covid 19
78
covid 19
79
covid 19
82
covid 19
83
covid19 data;covid 19
84
covid 19
85
covid 19
86
react;reactjs;covid 19
88
covid 19
89
covid 19
90
react;covid 19
92
coronavirus tracking;covid 19
93
covid 19
94
covid 19
95
covid 19
96
covid 19
97
covid 19
98
react;reactjs;typescript;covid 19
99
covid 19
101
covid 19
102
covid 19
104
covid 19
105
covid 19
107
covid 19
109
covid 19
110
covid 19
112
covid 19
113
covid 19
114
covid 19;python
115
covid 19
116
covid 19
117
covid 19
118
co

# 4 data analysis

In [1]:
'''
    标签抽取结果标签统计
'''
import csv
import numpy as np
import pandas as pd
from collections import Counter

with open(".\\Data in the analysis process\\extractionResult_SVM.csv",'r',encoding='UTF-8') as csvfile1:
    reader1 = csv.reader(csvfile1);
    result1= list(reader1);

all_tags = []

for line in result1:
    if str(line[1]) !="nan" and str(line[1]) !="":
        tags = str(line[8]).strip().split(";")
    all_tags = all_tags + tags
    
result = Counter(all_tags)
d = sorted(result.items(), key=lambda x: x[1], reverse=True)


tag = pd.DataFrame({"tag":list(list(zip(*d))[0]),"count":list(list(zip(*d))[1])}, columns=["tag","count"])

tag.to_csv(".\\Data in the analysis process\\tag_result.csv",index=False)
print("Finish!")

Finish!


3.1 Co word clustering

In [2]:
'''
    (1)生成共词矩阵
'''
import pandas as pd
import numpy as np
from collections import Counter
import math


#key_tag=['covid-19', 'javascript', 'python', 'jupyter-notebook', 'coronavirus-tracking', 'covid19-data', 'html', 'react', 'webapp', 'r', 'covid19-app', 'data-visualization', 'data-analysis', 'java', 'covid-api', 'typescript', 'covid-19-india', 'android-application', 'dart', 'deep-learning', 'pandemic', 'css', 'flutter', 'dashboard', 'statistics', 'coronavirus-real-time', 'vuejs', 'dataset', 'model', 'php', 'machine-learning', 'map', 'xray', 'kotlin', 'forecasting', 'coronavirus-info', 'simulation', 'npm', 'nodejs', 'covid-19-detection', 'twitter', 'health', 'data-science', 'coronavirus-analysis', 'angular', 'scraper', 'country', 'bot', 'plotly', 'infection', 'charts', 'swift', 'react-native', 'sir-model', 'brazil', 'ios', 'hackathon', 'c++', 'flask', 'open-source', 'spread', 'epidemiology', 'news', 'go', 'ruby', 'telegram-bot', 'deep-neural-networks', 'shell', 'outbreak', 'social-distancing', 'ct-scans', 'nlp', 'docker', 'material-ui', 'deaths', 'modeling', 'artificial-intelligence', 'graphics', 'computer-vision', 'monitoring', 'backend', 'time-series', 'sentiment-analysis', 'open-data', 'rest-api', 'mask', 'frontend', 'spring-boot', 'seir-model', 'chatbot', 'face-mask-detection', 'json', 'csv', 'database', 'pandas', 'cases', 'live', 'world', 'express', 'covid-19-italy', 'game', 'johns-hopkins-university', 'shiny', 'matlab', 'tensorflow', 'classification', 'firebase', 'covid19-dataset', 'italy', 'django', 'cli', 'beautifulsoup4', 'gatsby', 'ventilator', 'bootstrap', 'pneumonia', 'healthcare', 'quarantine', 'keras', 'mobile-app', 'script', 'wuhan', 'report', 'chartjs', 'nextjs', 'mobile-development', 'covid-19-data-analysis', 'coronavirus-globaloutbreak', 'simple', 'discord-bot', 'medical', 'mobile', 'china', 'code', 'golang', 'react-hooks', 'github-pages', 'convolutional-neural-networks', 'laravel', 'community', 'graphql', 'kaggle', 'hospital', 'heroku', 'c', 'covid-19-brazil', 'notification', 'lockdown', 'reactapp', 'matplotlib', 'neural-network', 'mexico', 'pytorch', 'bluetooth-low-energy', 'mongodb', 'chrome-extension', 'mvvm', 'create-react-app', 'wirvsvirus', 'analytics', 'list', 'canada', 'trend', 'government', 'google', 'resources', 'redux', 'retrofit2', 'france', 'free', 'aws', 'crawler', 'julia', 'tsql', 'tool', '3d', 'germany', 'mapbox', 'symptoms', 'project', 'ui', 'volunteers', 'pwa', 'challenge', 'dash', 'opencv', 'image-classification', 'arduino', 'interactive', 'jekyll', 'ios-app', 'tika-server', 'papers', 'bioinformatics', 'twitter-bot', 'android-studio', 'angular-cli9', 'leaflet', 'numpy', 'yarn', 'flutter-ui', 'swiftui', 'd3', 'discord', 'lstm', 'exposure-notification', 'kotlin-android', 'nigeria', 'education', 'demo', 'netlify', 'search', 'social', 'rstats', 'bangladesh', 'covid-tracking', 'ppe', 'uk', 'html-css-javascript', 'impact', 'transfer-learning', 'api-rest', 'online', 'covid-19-us', 'sri-lanka', 'testing', 'yolov3', 'delivery', 'axios', 'time-series-analysis', 'updates', 'colombia']

f = open(".\\Data in the analysis process\\data_analysis\\main_class.txt")
line = f.readline()
key_tag = []
while line:
    key_tag.append(line.replace("\n",""))
    line = f.readline()
f.close()

word_set = pd.read_csv(".\\Data in the analysis process\\extractionResult_SVM.csv")
word_set=word_set[~(word_set['topic'].isnull())]
#sametag = pd.read_csv("E:\\新冠数据\\数据530\\标签抽取\\标签共现分析\\同义替换.csv")
#sametagdic = dict(zip(sametag["label"].values, sametag["replace by"].values))
tag_set = pd.read_csv(".\\Data in the analysis process\\tag_result.csv")
tag_set_dic = dict(zip(tag_set["tag"].values, tag_set["count"].values))
#key_tag = tag_set[tag_set["count"]>1]["tag"].values.tolist()


table = np.zeros((len(key_tag),len(key_tag)))


bian_set = {}

for i in word_set["topic"].values:
    w = i.split(";")
    #for item in range(0,len(w)):
    #    if w[item] in sametagdic.keys():
    #        w[item]= sametagdic[w[item]]
    #w = list(set(w))
    for j in range(0,len(w)-1):
        if w[j] not in key_tag:continue
        for k in range(j+1,len(w)):
            if w[k] not in key_tag:continue
            table[key_tag.index(w[j])][key_tag.index(w[k])] +=1
            table[key_tag.index(w[k])][key_tag.index(w[j])] +=1

for m in range(0,len(key_tag)):
    table[m][m] = tag_set_dic[key_tag[m]]
    #table[m][m] = 0
    
network = pd.DataFrame(table)

network.columns = key_tag

network.to_csv(".\\Data in the analysis process\\data_analysis\\Co-occurrence-table-exist.csv",index = False)
print("Finish")

#基于Ochiia系数计算亲和矩阵
s_table = np.zeros((len(key_tag),len(key_tag)))
for j in range(0,len(key_tag)-1):
    print("正在处理……")
    for k in range(j+1,len(key_tag)):
        s = table[j][k]/(math.sqrt(table[j][j])*math.sqrt(table[k][k]))
        #s = table[j][k]/(table[j][j]+table[k][k]-table[j][k])
        s_table[j][k] = s
        s_table[k][j] = s

for m in range(0,len(key_tag)):
    s_table[m][m] = 1
    
s_network = pd.DataFrame(s_table)

s_network.columns = key_tag
s_network.to_csv(".\\Data in the analysis process\\data_analysis\\s-table-manual.csv",index = False)
print("Finish")

Finish
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
正在处理……
Finish


In [3]:
'''
    (2)使用sklearn进行谱聚类,导入共词矩阵，使用calinski_harabasz_score确定最佳聚类数
'''

import numpy as np
from matplotlib import pyplot as plt

from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation

import sklearn.metrics
import scipy.spatial.distance as dist
from scipy.cluster.hierarchy import dendrogram, linkage
import pandas as pd

s_table = pd.read_csv(".\\Data in the analysis process\\data_analysis\\s-table-manual.csv")
dist_matrix = s_table.values

#基于自定义的聚类矩阵进行聚类
def ac(k):
    y_pred = SpectralClustering(k, affinity='precomputed',assign_labels='discretize').fit_predict(dist_matrix)
# y_pred = AffinityPropagation(random_state=15,affinity='precomputed',preference=-1).fit_predict(dist_matrix)
# print(list(y_pred))
# print(list(y_pred).count(0))
    print(k)
    print("n_clusters=", k,"score:", sklearn.metrics.calinski_harabasz_score(dist_matrix, y_pred)) 

for k in range(2,16):
    ac(k)

2
n_clusters= 2 score: 4.00379643713501
3
n_clusters= 3 score: 3.6934464663118987
4
n_clusters= 4 score: 3.843796141863791
5
n_clusters= 5 score: 4.309070805471227
6
n_clusters= 6 score: 4.116053058456658
7
n_clusters= 7 score: 3.6429026294640665
8
n_clusters= 8 score: 3.7687850998561956
9
n_clusters= 9 score: 3.761125499419387
10
n_clusters= 10 score: 3.6383899231172134
11
n_clusters= 11 score: 3.620926173688063
12
n_clusters= 12 score: 3.6438016656578
13
n_clusters= 13 score: 3.3800463530515135
14
n_clusters= 14 score: 3.3577637378849574
15
n_clusters= 15 score: 3.366840551808138


In [4]:
'''
    (3)对主题标签进行谱聚类，并将聚类结果生成vosViwer能直接可视化的map和network格式
'''

import numpy as np
from matplotlib import pyplot as plt

from sklearn.cluster import SpectralClustering

import sklearn.metrics
import scipy.spatial.distance as dist
from scipy.cluster.hierarchy import dendrogram, linkage
import pandas as pd

#s_table = pd.read_csv("C:\\Users\\Administrator\\Desktop\\s-table-manual.csv")
#dist_matrix = s_table.values
otable = pd.read_csv(".\\Data in the analysis process\\data_analysis\\s-table-manual.csv")
ctable = otable.values
tag_set = pd.read_csv(".\\Data in the analysis process\\tag_result.csv")
tag_set_dic = dict(zip(tag_set["tag"].values, tag_set["count"].values))

# 基于自定义的聚类矩阵进行谱聚类discretize
y_pred = SpectralClustering(4, affinity='precomputed', assign_labels='discretize').fit_predict(ctable)
print(y_pred)
print("n_clusters=",4,"score:", sklearn.metrics.calinski_harabasz_score(ctable, y_pred)) 

tag = list(s_table.columns)

#生成vosViewer的map文件
vmap = pd.DataFrame(columns=['id','label','cluster','weight'])
biantable = pd.DataFrame(columns=["source","aim","w"])
for item in tag:
    index = tag.index(item)
    vmap = vmap.append({'id':index,'label':item,'cluster':y_pred[index]+1,'weight':tag_set_dic[key_tag[index]]},ignore_index=True)
vmap.to_csv(".\\Data in the analysis process\\data_analysis\\map-exist-1.csv",index = False)
    
for i in range(0,len(tag)-1):
    for j in range(i+1,len(tag)):
        if table[i][j]!=0:
            biantable = biantable.append({"source":i,"aim":j,"w":ctable[i][j]},ignore_index=True)
biantable.to_csv(".\\Data in the analysis process\\data_analysis\\network-exist-1.csv",index = False)
print("Finish!")

[1 1 1 0 0 1 1 0 0 1 1 3 1 2 1 2 3 0 1 1 1 1 1 1 1 1 1 1 1 3 3 1 0 0 0 0 0
 3 0 0 0 0 3 0 1 0 2 1 3 3 2 1 0 0 1 1 0 0 0 1 2 2 0 2 1 1 1 1 1 1 1 1 2 3
 1 1 1 0 0 1 1 1 1 0 2 1 1 1 0 3 1 0 0 3 2 0 3 0 1 1 0 0 1 1 0 2]
n_clusters= 4 score: 3.843796141863791
Finish!


3.2 Word frequency analysis

In [5]:
'''
    各类主题标签按月份统计平均词频
'''

import pandas as pd
import numpy as np

tag_cluster = pd.read_csv(".\\Data in the analysis process\\data_analysis\\tag_cluster.csv")

data = pd.read_csv(".\\Data in the analysis process\\extractionResult_SVM.csv")
data=data[~(data['topic'].isnull())]

result = pd.DataFrame(columns=["tag","cluster","mounth","count"])
tag_list = tag_cluster["tag"].values.tolist()
table = np.zeros((len(tag_list),9))

#从数据中提取出年和月
data["year"] = data['createTime'].apply(lambda x:x[:4])
data["mounth"] = data["createTime"].apply(lambda x:x[5:6])
data.head()
                 
for m in range(1,10):
    for line in data[(data["year"]=="2020") & (data["mounth"]==str(m))]["topic"].values:
        tags = line.split(";")
        for item in tags:
            if item in tag_list:
                 table[tag_list.index(item)][m-1] +=1


for t in range(0,len(tag_list)):
    for m in range(0,9):
        cluster = tag_cluster[tag_cluster["tag"]==tag_list[t]]["cluster"].values[0]
        result = result.append([{"tag":tag_list[t],"cluster":cluster,"mounth":m+1,"count":table[t][m]}])

result.head()
result.to_csv(".\\Data in the analysis process\\data_analysis\\Cluster label statistics.csv",index=False)

g = result.groupby(by=['cluster', 'mounth']).agg({'count':np.mean})
# 取消分组后的索引
h = g.reset_index()
h.to_csv(".\\Data in the analysis process\\data_analysis\\Category monthly average word frequency.csv",index=False)
print("Finish！")

Finish！


3.3 association analysis 

In [6]:
'''
    (1)加载数据
'''
import pandas as pd
def load_data(path):
    result=[]
    data = pd.read_csv(path)
    for line in data.values:
        #line=line.strip('\n')
        #split处理之后返回list，相当于二维数组的形式
        if str(line[8])=="nan":continue
        topics = line[8].split(";")
        if "covid 19" in topics:
            topics.remove("covid 19")
        if len(topics)>1:
            result.append(topics)
    return result
dataset=load_data(".\\Data in the analysis process\\extractionResult_SVM.csv")
print(len(dataset))
print(dataset[:10])

3593
[['python', 'nlp'], ['docker', 'django'], ['javascript', 'nodejs', 'api', 'coronavirus real time', 'coronavirus tracking', 'django'], ['pandemic', 'epidemiology', 'covid19 data', 'open data', 'epidemic'], ['vuejs', 'php', 'vue'], ['flutter', 'flutter apps'], ['python', 'visualization'], ['vuejs', 'covid19 data', 'vue'], ['coronavirus tracking', 'flutter apps', 'flutter', 'dart'], ['jupyter notebook', 'data science']]


In [7]:
'''
    (2)数据转码
'''
import itertools
items=set(itertools.chain(*dataset))
#用来保存字符串到编号的映射
str_to_index={}
#用来保存编号到字符串的映射
index_to_str={}
for index,item in enumerate(items):
    str_to_index[item]=index
    index_to_str[index]=item
#输出结果
print("字符串到编号:",list(str_to_index.items())[:5])
print("编号到字符串:",list(index_to_str.items())[:5])

#将原始数据进行转换，由字符串映射为数值索引
for i in range(len(dataset)):
    for j in range(len(dataset[i])):
        dataset[i][j]=str_to_index[dataset[i][j]]
for i in range(10):
    print(i+1,dataset[i],sep="->")

字符串到编号: [('covid19 dataset', 0), ('docker', 1), ('java', 2), ('data science', 3), ('vuejs', 4)]
编号到字符串: [(0, 'covid19 dataset'), (1, 'docker'), (2, 'java'), (3, 'data science'), (4, 'vuejs')]
1->[72, 68]
2->[1, 41]
3->[50, 11, 90, 47, 49, 41]
4->[44, 17, 87, 96, 15]
5->[4, 45, 42]
6->[16, 22]
7->[72, 76]
8->[4, 87, 42]
9->[49, 22, 16, 80]
10->[58, 3]


In [8]:
'''
    (3)生成候选1项集
'''
def buildC1(dataset):
    item1=set(itertools.chain(*dataset))
    return [frozenset([i]) for i in item1]
# c1=buildC1(dataset)
# c1

In [9]:
'''
    (4)根据候选k项集和最小支持度，生成频繁k项集
'''
def ck_to_lk(dataset,ck,min_support):
    support={}#定义项集-频数字典，用来存储每个项集key对应的频数value
    for row in dataset:
        for item in ck:
            #判断项集是否在记录中出现
            if item.issubset(row):
                support[item]=support.get(item,0)+1
    total=len(dataset)
    return {k:v/total for k,v in support.items() if v/total>=min_support}
# L1=ck_to_lk(dataset,c1,0.01)
# L1

In [10]:
'''
(5)频繁K项集组合生成候选K+1项集
'''
def lk_to_ck(lk_list):
    #保存所有组合之后的候选k+1项集
    ck=set()
    lk_size=len(lk_list)
    if lk_size>1:
        k=len(lk_list[0])
        for i,j in itertools.combinations(range(lk_size),2):
            t=lk_list[i]|lk_list[j]
            if len(t)==k+1:
                ck.add(t)
    return ck
# c2=lk_to_ck(list(L1.keys()))
# c2

# L2=ck_to_lk(dataset,c2,0.01)
# L2

In [11]:
'''
    (6)生成所有频繁项集，从原始数据生成频繁项集
'''
def get_L_all(dataset,min_support):
    c1=buildC1(dataset)
    L1=ck_to_lk(dataset,c1,min_support)
    L_all=L1
    Lk=L1
    while len(Lk)>1:
        lk_key_list=list(Lk.keys())
        ck=lk_to_ck(lk_key_list)
        Lk=ck_to_lk(dataset,ck,min_support)
        print("loading......")
        if len(Lk)>0:
            L_all.update(Lk)
        else:
            break
        if len(list(Lk.keys())[0])>=2:break
    return L_all
# L_all=get_L_all(dataset,0.02)
# L_all

In [12]:
'''
    (7)从频繁项集生成关联规则
''' 
def rules_from_item(item):
    #定义规则左侧的列表
    left=[]
    for i in range(1,len(item)):
        left.extend(itertools.combinations(item,i))
    return [(frozenset(le),frozenset(item.difference(le))) for le in left]
# rules_from_item(frozenset({1,2,3}))

In [13]:
'''
    (8)根据关联规则，计算置信度，保留符合最小置信度的关联规则
'''
def rules_from_L_all(L_all,min_confidence):
    #保存所有候选的关联规则
    rules=[]
    for lk in L_all:
        if len(lk)>1:
            rules.extend(rules_from_item(lk))
    result=[]
    for left,right in rules:
        support=L_all[left|right]
        confidence=support/L_all[left]
        lift=confidence/L_all[right]
        if confidence>=min_confidence:
            result.append({"左侧":left,"右侧":right,"支持度":support,"置信度":confidence,"提升度":lift})
    return result
# rules_from_L_all(L_all,0.05)

In [14]:
'''
    (9)最终程序,从原始数据生成关联规则
'''
def apriori(dataset,min_support,min_confidence):
    L_all=get_L_all(dataset,min_support)
    rules=rules_from_L_all(L_all,min_confidence)
    return rules
rules=apriori(dataset,0.002,0.2)
rules

loading......


[{'左侧': frozenset({68}),
  '右侧': frozenset({72}),
  '支持度': 0.0025048705816866127,
  '置信度': 0.2727272727272727,
  '提升度': 1.6552518427518426},
 {'左侧': frozenset({49}),
  '右侧': frozenset({47}),
  '支持度': 0.05733370442527136,
  '置信度': 0.24037339556592766,
  '提升度': 3.5541630052196624},
 {'左侧': frozenset({47}),
  '右侧': frozenset({49}),
  '支持度': 0.05733370442527136,
  '置信度': 0.8477366255144032,
  '提升度': 3.5541630052196624},
 {'左侧': frozenset({41}),
  '右侧': frozenset({49}),
  '支持度': 0.003339827442248817,
  '置信度': 0.2181818181818182,
  '提升度': 0.914734273894134},
 {'左侧': frozenset({50}),
  '右侧': frozenset({49}),
  '支持度': 0.01697745616476482,
  '置信度': 0.2040133779264214,
  '提升度': 0.855332633476817},
 {'左侧': frozenset({11}),
  '右侧': frozenset({50}),
  '支持度': 0.01224603395491233,
  '置信度': 0.3548387096774194,
  '提升度': 4.263998273815946},
 {'左侧': frozenset({90}),
  '右侧': frozenset({49}),
  '支持度': 0.015029223490119677,
  '置信度': 0.2755102040816327,
  '提升度': 1.155085371371419},
 {'左侧': frozenset({96}),
 

In [15]:
'''
    (10)把代码转换成真实的对象名称，同时为了能清晰呈现数据，用dataframe对象进行展示
'''
import pandas as pd
def change(item):
    li=list(item)
    for i in range(len(li)):
        li[i]=index_to_str[li[i]]
    return li

#读取核心主题标签
f = open(".\\Data in the analysis process\\data_analysis\\class1.txt")
line = f.readline()
key_tag = []
while line:
    key_tag.append(line.replace("\n",""))
    line = f.readline()
f.close()

clean_rules = pd.DataFrame(columns=["左侧","右侧","关联规则","支持度","置信度","提升度"])

df=pd.DataFrame(rules)
df=df.reindex(["左侧","右侧","支持度","置信度","提升度"],axis=1)
df["左侧"]=df["左侧"].apply(change)
df["右侧"]=df["右侧"].apply(change)

rule_list = []

for line in df.values:
    r = line[0]+line[1]
    flag = True
    for i in list(set(r)):
        if i not in key_tag:
            flag=False
    if flag==False:
        continue
    if set(r) in rule_list:
        continue
    else:
        rule_list.append(set(r))
        clean_rules = clean_rules.append([{"左侧":";".join(line[0]),
                                           "右侧":";".join(line[1]),
                                           "关联规则":";".join(line[0])+" => "+";".join(line[1]),
                                           "支持度":line[2],"置信度":line[3],"提升度":line[4]}],ignore_index=True)
df

df.to_csv(".\\Data in the analysis process\\data_analysis\\Association rules_1.csv",index=False)
clean_rules.to_csv(".\\Data in the analysis process\\data_analysis\\Association rules_clean_1.csv",index=False)
print("Finish!")

Finish!


In [16]:
'''
    将关联规则生成可视化文件
    (PS:此步生成的可视化文件需要打开Association rules_edge将表头删掉才能用vosViewer可视化)
'''
import pandas as pd

chart_data = pd.read_csv(".\\Data in the analysis process\\data_analysis\\Association rules_clean_1.csv")
class_data = pd.read_csv(".\\Data in the analysis process\\data_analysis\\map-exist-1.csv")

#点
node = pd.DataFrame(columns=["Id","Label","cluster","weight"])
#边
edge = pd.DataFrame(columns=["Source","Target","weight"])

n = {}
for line in chart_data.values:
    item = line[0].split(";")+line[1].split(";")
    for i in item:
        if i in n.keys():
            n[i]+=1
        else:
            n[i]=1
Id = 0
for item in n.keys():
    node = node.append([{"Id":Id,
                         "Label":item,
                         "cluster":class_data[class_data["label"]==item]["cluster"].values[0],
                         "weight":n[item]}],ignore_index=True)
    Id+=1
e = {}
for line in chart_data.values:
    left = line[0].split(";")
    right = line[1].split(";")
    for l in left:
        for r in right:
            ee1 = str(node[node["Label"]==l]["Id"].values[0])+";"+str(node[node["Label"]==r]["Id"].values[0])
            ee2 = str(node[node["Label"]==r]["Id"].values[0])+";"+str(node[node["Label"]==l]["Id"].values[0])
            if ee1 in e.keys():
                e[ee1]+=1
            elif ee2 in e.keys():
                e[ee2]+=1
            else:
                e[ee1]=line[5]
for item in e.keys():
    edge = edge.append([{"Source":item.split(";")[0],
                         "Target":item.split(";")[1],
                         "weight":e[item]}],ignore_index=True)
    
node.to_csv(".\\Data in the analysis process\\data_analysis\\Association rules_node_1.csv",index=False)
edge.to_csv(".\\Data in the analysis process\\data_analysis\\Association rules_edge_1.csv",index=False)
print("Finish!")

Finish!


In [17]:
pip freeze 

argon2-cffi==20.1.0
async-generator==1.10
attrs==20.2.0
backcall==0.2.0
bleach==3.2.1
boto3==1.15.14
botocore==1.18.14
certifi==2020.6.20
cffi==1.14.3
chardet==3.0.4
click==7.1.2
colorama==0.4.3
cycler==0.10.0
Cython==0.29.14
decorator==4.4.2
defusedxml==0.6.0
Distance==0.1.3
entrypoints==0.3
gensim==3.8.3
graphviz==0.14.2
idna==2.10
imageio==2.9.0
imbalanced-learn==0.7.0
imblearn==0.0
install==1.3.4
ipykernel==5.3.4
ipython==7.18.1
ipython-genutils==0.2.0
ipywidgets==7.5.1
jedi==0.17.2
jieba==0.42.1
Jinja2==2.11.2
jmespath==0.10.0
joblib==0.17.0
jsonschema==3.2.0
jupyter==1.0.0
jupyter-client==6.1.7
jupyter-console==6.2.0
jupyter-core==4.6.3
jupyterlab-pygments==0.1.2
kiwisolver==1.2.0
MarkupSafe==1.1.1
matplotlib==3.3.2
mistune==0.8.4
nbclient==0.5.0
nbconvert==6.0.7
nbformat==5.0.7
nest-asyncio==1.4.1
networkx==2.5
nltk==3.5
notebook==6.1.4
numpy==1.19.2
opencv-python==4.4.0.44
packaging==20.4
pandas==1.1.2
pandocfilters==1.4.2
parso==0.7.1
pickleshare==0.7.5
Pillow==7.2.0
prometheu